# This is a folk of [this notebook](https://www.kaggle.com/nyanpn/yet-another-leakage-lb0-995)
I just want to confirm that this notebook is not valid for the new dataset.

In [ ]:
!pip install nyaggle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nyaggle.experiment import run_experiment
from tqdm.notebook import tqdm
import os
%matplotlib inline


In [ ]:
train = pd.read_csv('/kaggle/input/seti-breakthrough-listen/train_labels.csv')
test = pd.read_csv('/kaggle/input/seti-breakthrough-listen/sample_submission.csv')

def get_file_path(image_id: str, train_or_test: str, data_dir: str = '/kaggle/input/seti-breakthrough-listen/') -> str:
    return os.path.join(data_dir, train_or_test, image_id[0], f"{image_id}.npy")

train['type'] = 'train'
test['type'] = 'test'
df = pd.concat([train, test]).reset_index(drop=True)

mean = []
std = []
minv = []
maxv = []
nuniq = []

for i, row in tqdm(df.iterrows()):
    arr = np.load(get_file_path(row.id, row.type))[[0,2,4],:,:].astype(float)
    mean.append(np.mean(arr))
    std.append(np.std(arr))
    minv.append(np.min(arr))
    maxv.append(np.max(arr))
    nuniq.append(len(np.unique(arr)))
    
df['mean'] = mean
df['std'] = std
df['min'] = minv
df['max'] = maxv
df['nuniq'] = nuniq

df.to_csv('seti-stats.csv', index=False)

## Visualize basic statistics

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16,7))

axes[0].set_xlim(-0.0001, 0.0001)
axes[0].set_title('mean of image')
axes[1].set_xlim(0.9998, 1.0002)
axes[1].set_title('std of image')

sns.distplot(df[df['target']==1]['mean'], label='target=1', ax=axes[0])
sns.distplot(df[df['target']==0]['mean'], label='target=0', ax=axes[0])

sns.distplot(df[df['target']==1]['std'], label='target=1', ax=axes[1])
sns.distplot(df[df['target']==0]['std'], label='target=0', ax=axes[1])

plt.legend()

Images with target=0 appear to be normalized to mean 0 and variance 1, whereas **images with target=1 are clearly seen to have a positive bias in the mean and a larger variance.**

Perhaps the host added the target signal to the image **after** normalizing the background noise. (Of course, to prevent leakage, the image should be normalized after the target signal is added.

## Modeling

In [ ]:
params = {
    'objective': 'binary',
    'metrics': 'auc',
    'n_estimators': 300
}
features = ['mean', 'std', 'min', 'max', 'nuniq']

X_train = df[df['type'] == 'train'][features]
y_train = df[df['type'] == 'train']['target']
X_test = df[df['type'] == 'test'][features]
result = run_experiment(params, X_train, y_train, X_test, fit_params={'verbose': 50})

In [ ]:
print(f'OOF SCORE: {result.metrics[-1]}')

A simple statistic alone was able to produce a CV=0.93. Perhaps the model for the participants at the top of the leaderboard implicitly takes advantage of this difference in data distribution.

In [ ]:
submission = pd.read_csv('/kaggle/input/seti-breakthrough-listen/sample_submission.csv')
submission['target'] = result.test_prediction
submission.to_csv('submission.csv', index=False)